In [1]:
folder = 'C:/Users/alire/OneDrive/data/asmahani_github/ordinal-boost/wine_quality'
filename_red, filename_white = 'winequality-red.csv', 'winequality-white.csv'

import os
import pandas as pd

def load_data(folder, filename):
    csv_path = os.path.join(folder, filename)
    return pd.read_csv(csv_path, sep = ';')

red_wine = load_data(folder, filename_red)
white_wine = load_data(folder, filename_white)

red_wine['quality'] = red_wine['quality'] - 3
white_wine['quality'] = white_wine['quality'] - 3

print(f'Shape of red wine: {red_wine.shape}')
print(f'Shape of white wine: {white_wine.shape}')

red_wine.head()
#print(white_wine.head())

Shape of red wine: (1599, 12)
Shape of white wine: (4898, 12)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,2
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,2
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,2


In [2]:
X_red, y_red = red_wine.drop(columns = ['quality']).to_numpy(), red_wine['quality'].to_numpy(dtype = 'int')
X_white, y_white = white_wine.drop(columns = ['quality']).to_numpy(), white_wine['quality'].to_numpy(dtype = 'int')

In [3]:
from gbor.main import BoostedOrdinal
gbor = BoostedOrdinal()
gbor.get_params()

{'base_learner__ccp_alpha': 0.0,
 'base_learner__criterion': 'squared_error',
 'base_learner__max_depth': 3,
 'base_learner__max_features': None,
 'base_learner__max_leaf_nodes': None,
 'base_learner__min_impurity_decrease': 0.0,
 'base_learner__min_samples_leaf': 1,
 'base_learner__min_samples_split': 2,
 'base_learner__min_weight_fraction_leaf': 0.0,
 'base_learner__monotonic_cst': None,
 'base_learner__random_state': None,
 'base_learner__splitter': 'best',
 'base_learner': DecisionTreeRegressor(max_depth=3),
 'lr_g': 0.1,
 'lr_theta': 0.001,
 'max_iter': 100,
 'n_class': None,
 'n_iter_no_change': None,
 'reltol': 0.01,
 'validation_fraction': 0.1,
 'validation_stratify': True}

In [4]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform

my_grid = dict(base_learner__max_depth = [3, 4, 5, 6, 7, 8, 9],
            #lr_g = [0.01, 0.1, 0.5, 1],
            lr_g = loguniform(1e-4, 1e-1),
)
#obj = RandomizedSearchCV(BoostedOrdinal(), my_grid)

In [5]:
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import RepeatedStratifiedKFold, RepeatedKFold, KFold
#my_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2,
#    random_state=123)
my_cv = RepeatedKFold(n_splits=10, n_repeats=3,
    random_state=123
)
#my_cv = KFold(n_splits=5, shuffle=True, 
#              random_state=123
#              )
#my_cv = 10

#my_scoring = 'accuracy'
my_scoring = 'neg_mean_absolute_error'

In [6]:
scores_red_baseline = cross_val_score(
    BoostedOrdinal(), 
    X_red, y_red, cv = my_cv, 
    scoring = my_scoring,
    n_jobs = -1
)
scores_red_baseline.mean()

-0.411082285115304

In [7]:
scores_red_hpt = cross_val_score(
    RandomizedSearchCV(BoostedOrdinal(), my_grid, scoring=my_scoring), 
    X_red, y_red, cv = my_cv, 
    scoring = my_scoring,
    n_jobs = -1
)
scores_red_hpt.mean()

-0.38961084905660376

In [8]:
# t-test of the two models
from scipy.stats import ttest_rel
ttest_rel(scores_red_baseline, scores_red_hpt)

TtestResult(statistic=-2.44206620197967, pvalue=0.020935189179706274, df=29)